In [1]:
import matplotlib.pyplot as plt
import pickle
import numpy as np

In [2]:
data = open('data/UHTelPCC.pkl', 'rb')

In [3]:
data = pickle.load(data, encoding='latin-1')

In [4]:
(train_x,train_y), (val_x,val_y), (test_x,test_y) = data

In [5]:
train_y

array([  1,   1,   1, ..., 507, 507, 507], dtype=uint16)

In [6]:
req_labels = [1, 184, 284, 218, 229, 262, 240, 11, 98, 2, 5, 6, 8, 9, 195, 426, 173, 140, 204, 162, 38, 320, 151, 404]

In [7]:
n_train_x, n_train_y = list(), list()
n_val_x, n_val_y = list(), list()
n_test_x, n_test_y = list(), list()

for val, label in zip(train_x, train_y):
    if label in req_labels:
        n_train_x += [val]
        n_train_y += [label]
        
for val, label in zip(val_x, val_y):
    if label in req_labels:
        n_val_x += [val]
        n_val_y += [label]
        
for val, label in zip(test_x, test_y):
    if label in req_labels:
        n_test_x += [val]
        n_test_y += [label]

num = [len(set(n_train_y)) == len(req_labels), len(set(n_val_y)) == len(req_labels), 
       len(set(n_test_y)) == len(req_labels)]
n_train_x = np.array(n_train_x)
n_train_y = np.array(n_train_y)
n_val_x = np.array(n_val_x)
n_val_y = np.array(n_val_y)
n_test_x = np.array(n_test_x)
n_test_y = np.array(n_test_y)

print(n_train_x.shape, n_val_x.shape, n_test_x.shape, num)

(11324, 1024) (2229, 1024) (2229, 1024) [True, True, True]


In [8]:
dump = [(n_train_x,n_train_y), (n_val_x,n_val_y), (n_test_x,n_test_y)]
f = open('data/chars.pkl', 'wb')
pickle.dump(dump, f)
f.close()

In [9]:
f = open('data/chars.pkl', 'rb')
n_data = pickle.load(f)

In [10]:
(train_x,train_y), (val_x,val_y), (test_x,test_y) = n_data

In [11]:
if (val_x == test_x).all():
    print(Yes)

In [12]:
count = np.array([np.count_nonzero(train_y == x) for x in req_labels])
count.max(), count.min()

(1716, 8)

In [13]:
count

array([ 563, 1716,  436,  411,  573,  932, 1115,  109,  296,  175,  117,
          8,  338,   70, 1148,   28,  386,  315,  257,  747, 1143,  253,
        169,   19])

In [14]:
count_val = np.array([np.count_nonzero(val_y == x) for x in req_labels])
count_test = np.array([np.count_nonzero(test_y == x) for x in req_labels])

In [15]:
count_val

array([120, 305,  93,  87, 122, 174, 207,  23,  63,  37,  25,   1,  72,
        15, 214,   5,  82,  67,  54, 159, 213,  53,  35,   3])

In [16]:
count_test

array([120, 305,  93,  87, 122, 174, 207,  23,  63,  37,  25,   1,  72,
        15, 214,   5,  82,  67,  54, 159, 213,  53,  35,   3])

In [17]:
print(count_val == count_test); np.random.choice(3, 5)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True]


array([2, 2, 0, 2, 2])

In [18]:
def numbers(c):
    print(c[c > 30].shape)
    print(c[c > 50].shape)    
    print(c[c > 100].shape)    
    print(c[c > 150].shape)    
    print(c[c > 200].shape)
    
def numbers1(c):
    print(c[c > 10].shape)
    print(c[c > 20].shape)    
    print(c[c > 50].shape)    
    print(c[c > 80].shape)    
    print(c[c > 100].shape)
    
numbers(count)
print()
numbers1(count_val)
print()
numbers1(count_test)

(21,)
(21,)
(20,)
(18,)
(16,)

(21,)
(20,)
(16,)
(11,)
(8,)

(21,)
(20,)
(16,)
(11,)
(8,)


In [19]:
def select_subset(size, arr):
    # Select at random, duplicate the ones that are not enough
    if size <= arr.shape[0]:
        return arr[np.random.choice(arr.shape[0], size, replace=False)]
    else:
        return arr[np.random.choice(arr.shape[0], size)]

In [20]:
n_train_x[n_train_y == req_labels[0]].shape
select_subset(30, n_train_x[n_train_y == req_labels[0]]).shape

(30, 1024)

In [26]:
def create_dataset(data, labels, size):
    arr = []
    for i in range(len(req_labels)):
        arr.append(select_subset(size, data[labels == req_labels[i]]))
    return np.array(arr)
    

create_dataset(n_val_x, n_val_y, 30).shape

(24, 30, 1024)

In [22]:
len(req_labels)

24

In [34]:
for size1, size2 in zip([30, 50, 100, 150, 200], [10, 20, 50, 80, 100]):
    train = create_dataset(n_train_x, n_train_y, size1)
    val = create_dataset(n_val_x, n_val_y, size2)
    test = create_dataset(n_test_x, n_test_y, size2)
    print(train.shape, val.shape, test.shape)
    f = open('data/supervised/chars_{}.pkl'.format(size), 'wb')
    pickle.dump((train, val, test), f)
    f.close()

(24, 30, 1024) (24, 10, 1024) (24, 10, 1024)
(24, 50, 1024) (24, 20, 1024) (24, 20, 1024)
(24, 100, 1024) (24, 50, 1024) (24, 50, 1024)
(24, 150, 1024) (24, 80, 1024) (24, 80, 1024)
(24, 200, 1024) (24, 100, 1024) (24, 100, 1024)


In [25]:
from itertools import combinations

chars_labels = [(x, y) for x, y in list(combinations(req_labels, 2))]

In [33]:
def create_siamese(data, labels, size):
    arr = []
    for (x, y) in chars_labels:
        temp = np.array([select_subset(size, data[labels == x]), select_subset(size, data[labels == y])])
        arr.append(temp)
    return np.array(arr)

create_siamese(n_val_x, n_val_y, 30).shape

(276, 2, 30, 1024)

In [35]:
276 * 30

8280

In [43]:
x = 8279 // 30; y = 8219 // 276; print(x, y)

275 29


In [44]:
for size1, size2 in zip([30, 50, 100, 150, 200], [10, 20, 50, 80, 100]):
    train = create_siamese(n_train_x, n_train_y, size1)
    val = create_siamese(n_val_x, n_val_y, size2)
    test = create_siamese(n_test_x, n_test_y, size2)
    print(train.shape, val.shape, test.shape)
    f = open('data/siamese/chars_{}.pkl'.format(size), 'wb')
    pickle.dump((train, val, test), f)
    f.close()

(276, 2, 30, 1024) (276, 2, 10, 1024) (276, 2, 10, 1024)
(276, 2, 50, 1024) (276, 2, 20, 1024) (276, 2, 20, 1024)
(276, 2, 100, 1024) (276, 2, 50, 1024) (276, 2, 50, 1024)
(276, 2, 150, 1024) (276, 2, 80, 1024) (276, 2, 80, 1024)
(276, 2, 200, 1024) (276, 2, 100, 1024) (276, 2, 100, 1024)


In [47]:
from scipy.spatial.distance import squareform

a = np.arange(25)
ap = np.array([x * y for x, y in list(combinations(req_labels + [500], 2))])
print(ap.shape)
am = squareform(ap)
n_am = np.delete(am, 9, 0)
n_am = np.delete(n_am, 9, 1)
squareform(n_am).shape

(300,)


(276,)